In [1]:
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kinesis import KinesisUtils, InitialPositionInStream
from pyspark.sql.types import *
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

import json

In [2]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName("Reddit-News")\
                    .getOrCreate()

In [3]:
pythonSchema = StructType() \
          .add("id", StringType(), True) \
          .add("submission", StringType(), True) \
          .add("comment_number", IntegerType(), True) \
          .add("score", IntegerType(), True)

In [4]:
awsAccessKeyId = "" 
awsSecretKey = ""  
kinesisStreamName = "" 
kinesisRegion = ""

In [5]:
kinesisDF = spark \
  .readStream \
  .format("kinesis") \
  .option("streamName", kinesisStreamName)\
  .option("region", kinesisRegion) \
  .option("initialPosition", "latest") \
  .option("format", "json") \
  .option("awsAccessKey", awsAccessKeyId)\
  .option("awsSecretKey", awsSecretKey) \
  .option("inferSchema", "true") \
  .load()

In [6]:
# Data sourced from kinesis will come into the Spark Streaming DF here
df = kinesisDF \
  .writeStream \
  .format("memory") \
  .outputMode("append") \
  .queryName("news")  \
  .start()

In [7]:
df.status

Out[18]: {'message': 'Getting offsets from KinesisV2[reddit_news]',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [ ]:
#sentiment
news = spark.sql("select cast(data as string) from news")
